In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import RepeatedStratifiedKFold, StratifiedShuffleSplit, train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import RFECV
from scipy.stats import uniform

from utils import impute, plot_scatter_matrix, plot_cross_validation, plot_hyperparameter_tuning

## 1. Load the Data

In [16]:
df=pd.read_parquet('../data/wrangled_df.parquet')

In [9]:
df.head(5)

,Local Event Date,Event City,Event State,Event Airport,Aircraft Damage,Flight Phase,Aircraft Model,Aircraft Registration Nbr,Total Fatalities,Total Injuries,Nbr of Engines,PIC Certificate Type,PIC Flight Time Total Hrs,PIC Flight Time Total Make-Model,Aircraft Make-Model,Phase Category
0,1978-01-01,WAHPETON,ND,BRECKENRIDGE,MINOR,ROLL-OUT (FIXED WING),182,2691Q,0,0,1.0,PRIVATE PILOT,245.0,136.0,CESSNA 182,LANDING
1,1978-01-01,FAIRBANKS,AK,FAIRBANKS INTL,MINOR,ROLL-OUT (FIXED WING),PA18,4073E,0,0,1.0,STUDENT,200.0,2.0,PIPER PA18,LANDING
3,1978-01-01,CARLSBAD,CA,MC CLELLAN-PALOMAR,MINOR,LEVEL OFF TOUCHDOWN,310,2250F,0,0,2.0,COMMERCIAL PILOT FLIGHT INSTRUCTOR,2000.0,400.0,CESSNA 310,LANDING
4,1978-01-01,TROUTDALE,OR,TROUTDALE MUNI,MINOR,"GROUND TAXI, OTHER AIRPLANE",172,738FD,0,0,1.0,COMMERCIAL PILOT,300.0,130.0,CESSNA 172,TAXIING
10,1978-01-02,LANGDON,ND,LANGDON MUNI,MINOR,TOUCH AND GO LANDING,AA1,6335L,0,0,1.0,STUDENT,29.0,21.0,GRUMMAN AA1,LANDING


In [17]:
df=df.rename(columns={'PIC Flight Time Total Hrs': 'target'})

In [11]:
df.head(5)

,Local Event Date,Event City,Event State,Event Airport,Aircraft Damage,Flight Phase,Aircraft Model,Aircraft Registration Nbr,Total Fatalities,Total Injuries,Nbr of Engines,PIC Certificate Type,target,PIC Flight Time Total Make-Model,Aircraft Make-Model,Phase Category
0,1978-01-01,WAHPETON,ND,BRECKENRIDGE,MINOR,ROLL-OUT (FIXED WING),182,2691Q,0,0,1.0,PRIVATE PILOT,245.0,136.0,CESSNA 182,LANDING
1,1978-01-01,FAIRBANKS,AK,FAIRBANKS INTL,MINOR,ROLL-OUT (FIXED WING),PA18,4073E,0,0,1.0,STUDENT,200.0,2.0,PIPER PA18,LANDING
3,1978-01-01,CARLSBAD,CA,MC CLELLAN-PALOMAR,MINOR,LEVEL OFF TOUCHDOWN,310,2250F,0,0,2.0,COMMERCIAL PILOT FLIGHT INSTRUCTOR,2000.0,400.0,CESSNA 310,LANDING
4,1978-01-01,TROUTDALE,OR,TROUTDALE MUNI,MINOR,"GROUND TAXI, OTHER AIRPLANE",172,738FD,0,0,1.0,COMMERCIAL PILOT,300.0,130.0,CESSNA 172,TAXIING
10,1978-01-02,LANGDON,ND,LANGDON MUNI,MINOR,TOUCH AND GO LANDING,AA1,6335L,0,0,1.0,STUDENT,29.0,21.0,GRUMMAN AA1,LANDING


## 2. Decision Tree

- Determine risk based on PIC Flight Time Total Hrs
- Then determine risk based on PIC Flight Time Total Make-Model? Compare to see if Total Hrs or Make-Model is more relevant?

In [18]:
X = df.drop('target', axis=1)
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)